In [1]:
import os 
import sys
import glob
import requests as req
import re
import pprint

In [2]:
import collections
import functools
import itertools
from itertools import product

In [3]:
import numpy as np
import pandas as pd
import numba as nb
import numexpr as ne
from scipy.special import gamma, binom

In [4]:
from IPython.display import display

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [7]:
def download_file(url, file, chunk_size=8192):
    """Download from 'url' save into 'file'."""
    with req.get(url, stream=True) as r:
        r.raise_for_status()
        with open(file, "wb") as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
                    f.flush()
    return file

In [8]:
# download all daily observations

for x in ("confirmed", "deaths", "recovered"):
    url = f"https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{x}_global.csv"
    file = f"time_series_covid19_{x}_global.csv"
    download_file(url, file)

In [9]:
! ls *.csv

time_series_covid19_confirmed_global.csv
time_series_covid19_deaths_global.csv
time_series_covid19_recovered_global.csv


In [10]:
# load all data as is

CSV = {
    c.split("/")[-1].split("_")[-2]: pd.read_csv(c) for c in glob.iglob(f"time_series_covid19*global.csv")
}

In [11]:
POPULATION = pd.read_excel(
    "population_per_country.xlsx", index_col="Name", usecols=["Name", "QTY"]
)
POPULATION.dropna(inplace=True)

In [12]:
# Clean up and pre-process

RENAME = {
    "Burma": "Myanmar",
    "Cabo Verde": "Cape Verde",
    "Congo (Brazzaville)": "Republic of the Congo",
    "Congo (Kinshasa)": "Democratic Republic of the Congo",
    "Cote d'Ivoire": "Ivory Coast",
    "Czechia": "Czech Republic",
    "Korea, South": "South Korea",
    "Sao Tome and Principe": "São Tomé and Príncipe",
    "Taiwan*": "Taiwan",
    "Timor-Leste": "East Timor",
    "US": "United States",
}

DATA = []

ndays = None
ncountries = None

for stat, df in CSV.items():

    # remove useless rows if present
    if len({"Lat", "Long", "Province/State"} - set(df.columns)) == 0:
        del (
            df["Lat"],
            df["Long"],
            df["Province/State"],
        )

    # rename some countries
    for orig, new in RENAME.items():
        df.loc[df["Country/Region"] == orig, "Country/Region"] = new

    # sum up all provinces and transpose
    df = df.groupby("Country/Region").sum().T

    # rename columns
    df.columns.name = "Country"
    ncountries = len(df.columns)

    # remove non countires
    if len({"Diamond Princess", "Holy See", "MS Zaandam"} - set(df.columns)) == 0:
        del (
            df["Diamond Princess"],
            df["Holy See"],
            df["MS Zaandam"],
        )

    # Add cum days
    df["Days"] = np.arange(1, len(df) + 1)
    ndays = len(df)

    # add Stat type Column
    df["StatType"] = stat

    # merge all into one df
    DATA.append(df)

DATA = pd.concat(DATA)

In [13]:
DATA.sample(5)

Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe,Days,StatType
3/31/20,174,243,716,376,7,7,1054,532,4559,10180,...,172,135,212,119,0,0,35,8,70,confirmed
2/23/20,0,0,0,0,0,0,0,0,15,0,...,0,0,16,0,0,0,0,0,33,confirmed
4/11/20,18,23,275,26,2,2,83,13,57,337,...,4,9,0,2,0,0,2,3,81,deaths
4/7/20,423,383,1468,545,17,19,1628,853,5895,12639,...,520,165,249,261,4,0,39,11,77,confirmed
3/5/20,1,0,12,1,0,0,1,1,55,41,...,0,0,16,4,0,0,0,0,44,confirmed


In [14]:
data = DATA.melt(id_vars=["Days", "StatType"], value_name="NumCases")

In [15]:
data.sample(5)

,Days,StatType,Country,NumCases
51172,54,recovered,Venezuela,0
44113,76,deaths,South Korea,186
28177,48,recovered,Lithuania,0
18464,35,deaths,Ghana,0
32077,68,confirmed,Montenegro,85


# Select Countries and Stats of Interest

In [16]:
Countries = [
    "Spain",
    "Netherlands",
    "Italy",
    "South Korea",
    "Germany",
    "Uzbekistan",
    "United States",
#     "Malaysia",
    "Russia",
#     "Kazakhstan",
    "South Korea",
    "Japan",
    "Norway",
    "New Zealand",
    "Australia",
    "France",
#     "Thailand",
#     "Vietnam",
    "United Kingdom",
    "China",
#     "Venezuela",
    "Singapore",
    "Hong Kong",
    "South Africa"
]
StatType = ["confirmed", "deaths", "recovered"]

In [17]:
# Sort acording to # cases

_latest = data.groupby("Country").max()[["NumCases"]]
Countries = (
    _latest[_latest.index.isin(Countries)]
    .sort_values("NumCases", ascending=False)
    .index.values
)

In [18]:
D = data[data["Country"].isin(Countries)]

In [19]:
selection = alt.selection_multi(fields=["Country"], bind="legend")
highlight = alt.selection_multi(on="mouseover", fields=["Country"], nearest=True)

In [20]:
base = (
    alt.Chart(D)
    .transform_filter(alt.FieldEqualPredicate(field="StatType", equal="confirmed"))
    .encode(
        alt.X("Days:Q"),
        alt.Y("NumCases:Q", scale=alt.Scale(type="symlog")),
        color="Country:N",
    )
)

points = base.mark_circle().encode(opacity=alt.value(0)).properties(selection=highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.3)),
)

points + lines

alt.LayerChart(...)

In [21]:
DERIV = data.copy()

grad = np.gradient(
    data["NumCases"].values.reshape(ndays, len(data) // ndays), axis=1
).reshape(len(data))
grad[grad < 0] = np.nan

DERIV.loc[:, "∂NumCases"] = grad

DE = DERIV[DERIV["Country"].isin(Countries)]

In [22]:
alt.Chart(DE).mark_line().encode(
    alt.X("Days:Q"),
    alt.Y("∂NumCases:Q", scale=alt.Scale(type="symlog")),
    color=alt.Color("Country", sort=Countries),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    row="StatType",
    order="Days",
).add_selection(selection)

alt.Chart(...)

In [23]:
selection = alt.selection_single(bind="legend", fields=["Country"], empty="none")

base = alt.Chart(DE).transform_filter("datum.StatType == 'confirmed'")

chart = (
    base.mark_line()
    .encode(
        alt.X("NumCases:Q", scale=alt.Scale(type="symlog")),
        alt.Y("∂NumCases:Q", scale=alt.Scale(type="symlog")),
        color=alt.Color("Country", sort=Countries),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        size=alt.condition(selection, alt.value(2), alt.value(1)),
        #     row="StatType",
        order="Days",
        tooltip="NumCases",
    )
    .add_selection(selection)
)

chart

alt.Chart(...)

In [24]:
selection = alt.selection_multi(bind="legend", fields=["Country"], empty="none")

base = (
    alt.Chart(DE)
    .mark_line()
    .encode(
        alt.Y("∂NumCases:Q", scale=alt.Scale(type="symlog")),
        color=alt.Color("Country", sort=Countries),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        order="Days",
    )
    .add_selection(selection)
    .properties(width=250, height=250)
)

confirmed = base.transform_filter("datum.StatType == 'confirmed'")
deaths = base.transform_filter("datum.StatType == 'deaths'")


confvsdays = confirmed.encode(alt.X("Days:Q"))

confvscases = confirmed.encode(alt.X("NumCases:Q", scale=alt.Scale(type="symlog")))

deathvsdays = deaths.encode(alt.X("Days:Q"))

deathvscases = deaths.encode(alt.X("NumCases:Q", scale=alt.Scale(type="symlog")))


(confvsdays | confvscases).properties(title="Confirmed") & (
    deathvsdays | deathvscases
).properties(title="Deaths")

alt.VConcatChart(...)

In [27]:
TODO all below

SyntaxError: invalid syntax (<ipython-input-27-de016a83f3b3>, line 1)

# Normalize to the total country population 

In [ ]:
fig, axes = plt.subplots(len(StatType), figsize=(8, 8 * len(StatType)))

NORM = pd.DataFrame(DATA[Countries])

for i, case in enumerate(StatType):

    for c in Countries:
        xc = ((DATA[c, case] / POPULATION.loc[c, "QTY"]) * 100).values
        yc = np.gradient(xc, axis=0)
        NORM[c, case] = xc
        NORM[c, f"D({case})"] = yc

    X = NORM[product(Countries, [case])]
    Y = NORM[product(Countries, [f"D({case})"])]

    # plot
    ax = axes[i]
    ax.set_title(f"{case}: ∂_% vs %population")

    ax.plot(X, Y, "o")

    ax.set_xscale("log")
    ax.set_xlabel("% population")

    ax.set_yscale("log")
    ax.set_ylabel("∂(% population)")

    ax.legend(Countries)

In [ ]:
# loglog per country pairs of combinations of [confirmed, deaths, recovered]

combi = list(itertools.combinations(StatType, 2))
fig, axes = plt.subplots(len(combi), figsize=(8, 8*len(combi)))
plt.tight_layout(3)

for i, (xn, yn) in enumerate(combi):
    ax = axes[i]
    
    for ic, cnt in enumerate(Countries):
        x = DATA[cnt, xn]
        y = DATA[cnt, yn]
        ax.plot(x, y)


    ax.set_xlabel(xn)
    ax.set_xscale("log")

    ax.set_ylabel(yn)
    ax.set_yscale("log")

    ax.set_title(f"{yn} vs {xn}")
    ax.legend(Countries)

In [ ]:
# loglog country1 vs country2

ncomb = int(binom(len(Countries), 2))
fig, axes = plt.subplots(ncomb, len(StatType), figsize=(10, ncomb * 5))
plt.tight_layout(3)

for i, stat in enumerate(StatType):
    for ic, (xn, yn) in enumerate(itertools.combinations(Countries, 2)):
        x = DATA[xn, stat]
        y = DATA[yn, stat]
        ax = axes[ic, i]
        ax.plot(x, y)

        ax.set_xscale("log")
        ax.set_xlabel(xn)

        ax.set_yscale("log")
        ax.set_ylabel(yn)

        ax.set_title(f"{stat}: {yn}({xn})")

# WORLD

In [ ]:
# ln(∂_cases) vs ln_cases

fig, axes = plt.subplots(len(StatType), figsize=(8, 8 * len(StatType)))

for i, stat in enumerate(StatType):
    # WORLD
    sel = product({x[0] for x in DATA.columns}, [stat])
    xw = DATA[sel].sum(axis=1).values
    yw = np.gradient(xw)

    # Except China
    no_china = product({x[0] for x in DATA.columns} - {"China"}, [stat])
    xnc = DATA[no_china].sum(axis=1).values
    ync = np.gradient(xnc)

    # plot
    ax = axes[i]
    ax.set_title(f"{stat}: ∂_cases vs cases")

    ax.plot(xw, yw, "-")
    ax.plot(xnc, ync, ".")

    ax.set_xscale("log")
    ax.set_xlabel("cases")

    ax.set_yscale("log")
    ax.set_ylabel("∂(cases)")

    ax.legend([f"World {xw[-1]:,}", f"NoChina {xnc[-1]:,}"])

# Merge Territories

In [ ]:
Europe = {
    "Austria",
    "Belgium",
    "Bulgaria",
    "Croatia",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Hungary",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Sweden",
    "Switzerland",
    'United Kingdom',
}

In [ ]:
Europe - {x[0] for x in DATA.columns}

In [ ]:
Africa = set(
    "Algeria | Angola | Benin | Botswana | Burkina Faso | Burundi | Cameroon | Cape Verde | Central African Republic | Chad | Comoros | Democratic Republic of the Congo | Djibouti | Egypt | Equatorial Guinea | Eritrea | Ethiopia | Gabon | Gambia | Ghana | Guinea | Guinea-Bissau | Ivory Coast | Kenya | Lesotho | Liberia | Libya | Madagascar | Malawi | Mali | Mauritania | Mauritius | Mayotte (overseas collectivity of France) | Morocco | Mozambique | Namibia | Niger | Nigeria | Republic of the Congo | Réunion (overseas region of France) | Rwanda | Saint Helena, Ascension and Tristan da Cunha (British overseas territory) | São Tomé and Príncipe | Senegal | Seychelles | Sierra Leone | Somalia | South Africa | South Sudan | Sudan | Eswatini | Tanzania | Togo | Tunisia | Uganda | Western Sahara | Zambia | Zimbabwe".split(
        " | "
    )
)

Africa - {x[0] for x in DATA.columns}

In [ ]:
MiddleEast = {
    "Bahrain",
    "Cyprus",
    "Egypt",
    "Gaza Strip",
    "Iran",
    "Iraq",
    "Israel",
    "Jordan",
    "Kuwait",
    "Lebanon",
    "Oman",
    "Qatar",
    "Saudi Arabia",
    "Syria",
    "Turkey",
    "United Arab Emirates",
    "West Bank",
    "Yemen",
}

MiddleEast - {x[0] for x in DATA.columns}

In [ ]:
SouthAmerica = set(
    "Argentina | Bolivia | Brazil | Chile | Colombia | Ecuador | Falkland Islands (British overseas territory) | French Guiana (French overseas region) | Guyana | Paraguay | Peru | Suriname | Uruguay | Venezuela".split(
        " | "
    )
)

SouthAmerica - {x[0] for x in DATA.columns}

In [ ]:
SouthEastAsia = {
    "Brunei",
    "Cambodia",
    "East Timor",
    "Indonesia",
    "Laos",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Vietnam",
}

SouthEastAsia - {x[0] for x in DATA.columns}

In [ ]:
CentralAsia = {"Kazakhstan", "Kyrgyzstan", "Tajikistan", "Turkmenistan", "Uzbekistan"}

CentralAsia - {x[0] for x in DATA.columns}

In [ ]:
EasternEurope = {
    "Belarus",
    "Bulgaria",
    "Czech Republic",
    "Hungary",
    "Moldova",
    "Poland",
    "Romania",
    "Slovakia",
    "Ukraine",
}

EasternEurope - {x[0] for x in DATA.columns}

In [ ]:
NorthAmerica = set(
    "Antigua and Barbuda | Bahamas | Barbados | Belize | Bermuda | British Virgin Islands | Canada | Cayman Islands | Costa Rica | Cuba | Curacao | Dominica | Dominican Republic | El Salvador | Greenland | Grenada | Guadeloupe | Guatemala | Haiti | Honduras | Jamaica | Martinique | Mexico | Montserrat | Nicaragua | Panama | Puerto Rico | Saint Kitts and Nevis | Saint Lucia | Saint Pierre and Miquelon | Saint Vincent and the Grenadines | Sint Maarten | Trinidad and Tobago | Turks and Caicos Islands | United States | United States Virgin Islands".split(
        " | "
    )
)

NorthAmerica - {x[0] for x in DATA.columns}

In [ ]:
Other = {x[0] for x in DATA.columns} - Europe - Africa - MiddleEast - SouthAmerica - SouthEastAsia - CentralAsia - EasternEurope - NorthAmerica

Other

In [ ]:
# merge larger territories
# "EU,Africa,MiddleEast,SouthAmerica,SouthEastAsia,CentralAsia,EasternEurope".split(",")

Countries_All = {x[0] for x in DATA.columns}

TERR = pd.DataFrame(index=DATA.index, columns=DATA.columns[:1])
TERR_PCT = pd.DataFrame(index=DATA.index, columns=DATA.columns[:1])

for stat in StatType:
    for terr, terr_set in zip(
        "Europe,Africa,MiddleEast,SouthAmerica,SouthEastAsia,CentralAsia,EasternEurope,NorthAmerica".split(
            ","
        ),
        (
            Europe,
            Africa,
            MiddleEast,
            SouthAmerica,
            SouthEastAsia,
            CentralAsia,
            EasternEurope,
            NorthAmerica,
        ),
    ):
        use_terr_set = Countries_All & terr_set
        TERR[terr, stat] = DATA[product(use_terr_set, [stat])].sum(axis=1)

        terr_popul = POPULATION.loc[use_terr_set, "QTY"].sum()
        TERR_PCT[terr, stat] = TERR[terr, stat] / terr_popul * 100

In [ ]:
# ln(∂_cases) vs ln_cases

Territories = [
    "Europe",
    "Africa",
    "MiddleEast",
    "SouthAmerica",
    "SouthEastAsia",
    "CentralAsia",
    "EasternEurope",
    "NorthAmerica",
]

fig, axes = plt.subplots(len(StatType), figsize=(8, 8 * len(StatType)))

DTERR = pd.DataFrame(
    np.gradient(TERR[Territories], axis=0), columns=TERR[Territories].columns
)

for i, stat in enumerate(StatType):
    X = TERR[product(Territories, [stat])]
    Y = DTERR[product(Territories, [stat])]
    ax = axes[i]
    ax.set_title(f"{stat}: ∂_cases vs cases")
    ax.plot(X, Y, "-")
    ax.set_xscale("log")
    ax.set_xlabel("cases")

    ax.set_yscale("log")
    ax.set_ylabel("∂(cases)")
    ax.legend(Territories)

In [ ]:
# % ln(∂_cases) vs ln_cases

Territories = [
    "Europe",
    "Africa",
    "MiddleEast",
    "SouthAmerica",
    "SouthEastAsia",
    "CentralAsia",
    "EasternEurope",
    "NorthAmerica",
]

fig, axes = plt.subplots(len(StatType), figsize=(8, 8 * len(StatType)))

DTERR_PCT = pd.DataFrame(
    np.gradient(TERR_PCT[Territories], axis=0), columns=TERR_PCT[Territories].columns
)

for i, stat in enumerate(StatType):
    X = TERR_PCT[product(Territories, [stat])]
    Y = DTERR_PCT[product(Territories, [stat])]
    ax = axes[i]
    ax.set_title(f"{stat}: ∂(% population) vs % population")
    ax.plot(X, Y, "-")
    ax.set_xscale("log")
    ax.set_xlabel("% population")

    ax.set_yscale("log")
    ax.set_ylabel("∂(% poplulation)")
    ax.legend(Territories)

In [ ]:
nodata = set(POPULATION.index) - {x[0] for x in DATA.columns}

In [ ]:
POPULATION.loc[nodata,:].sort_values(by="QTY")